In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# モジュールの読み込み
import sys
sys.path.append('./module')
from utils import make_lorenz96, rk4, error_series_kf, plot_error_KF, estimate_error_with_params

# EnSRF(ETKF)
from kalman_filters import EnsembleSquareRootFilter as EnSRF

In [ ]:
# Lorenz96の設定
J = 40
F = 8
lorenz = make_lorenz96(F)

# 同化step
# 時間発展は0.01ごとに行う
dt = 0.05

# モデルの遷移関数(非線形)
# 0.01ずつ時間発展させる
# dtは同化step
def M(x, dt):
    for i in range(int(dt/0.01)):
        x = rk4(0, x, 0.01, lorenz)
    return x

# 単位行列
I = np.identity(J)

# 観測
H = I

# モデル誤差共分散, 最初は完全モデルを仮定
Q = np.zeros((J, J))

# 観測誤差共分散, 後で定数倍の変化をさせる.
R = I

# 観測値と真値
end_step = 500 # 開発用
y = np.load('data/obs_atr.npy')
true = np.load('data/true_atr.npy')

# KFの初期値
np.random.seed(1)
x_0 = true[np.random.randint(len(true)-1)]
P_0 = 25*I

## Square Root Filter

観測に摂動を与えずに最小誤差共分散
$ P^a = (I - KH) P^f $を実現するような

$X^a = X^f T$を満たす線形変換$T$を考える．[三好](https://www.metsoc.jp/tenki/pdf/2005/2005_02_0093.pdf)

### Tの詳細
- $ dX^f = X^f - x^f_{mean} $
- $ dY = H dX^f $

とすると$ K = dX dY^T (dY dY^T + R)^{-1} $なので

$ T = \sqrt{I - dY^T (dYdY^T + R)^{-1} dY} $

とすれば良い．(直交変換の自由度があり一意的には定まらない)



### 別の実装 from Whitaker and Hamill (2002, MWR)
$ K = P^f H^T(H P^f H^T + R)^{-1} $に対して

$ K' = \alpha K$とおいて

$ dX^a = dX^f + K'(y^o - HdX^f)$

$ P^a = dX^a (dX^a)^T = (I-KH)P^f $となるように$\alpha $を調整する．

### パラメータチューニング
- N = 10 ~ 50, 10刻み
- $\alpha = 1 \sim 1.5, 0.05$刻み

### 20メンバーでのinflation factor $\alpha$のチューニング

In [ ]:
# %%time
# results = []
# params_alpha = []
# for k in range(10):
#     inf = 0.11 - 0.01 + 0.01*k
#     params_alpha.append(1+inf)
#     srf_inf = EnSRF(M, H, Q, R, y, x_0, P_0, N=20, alpha=1 + inf)
#     srf_inf.forward_estimation()
#     results.append(srf_inf.x)
# optimal_alpha, optimal_idx = estimate_error_with_params(true, results, params_alpha, 'alpha')
# np.save('data/srf/srf_optimal_200member.npy', results[optimal_idx])

### 観察
- SRFが20memberで動くことが確認できた．
- $\alpha = 1.12$ 程度
- 推定が落ち着くまでに100step以上かかっている．


### 精度を重視して50メンバーでのinflation factor $\alpha$のチューニング

In [ ]:
# %%time
# params_N = np.arange(1, 6)*10
# optimal_alpha_by_N = []
# params_alpha = []
# errors = np.zeros((20, 5))
# T = len(true) - 200
# for N in params_N:
#     results_srf_inf = []
#     for k in range(20):
#         inf = 0.005*k
#         if N==10:
#             params_alpha.append(1+inf)
#         srf_inf = EnSRF(M, H, Q, R, y, x_0, P_0, N=N, alpha=1 + inf)
#         srf_inf.forward_estimation()
#         results_srf_inf.append(srf_inf.x)
#         n = N//10 - 1
#         errors[k, n] = error_series_kf(true, srf_inf.x, np.arange(T)+200).mean()
#         print('complete: {}, {}'.format(N,1+inf))
#     optimal_alpha, optimal_idx, optimal_error = estimate_error_with_params(true, results_srf_inf, params_alpha, 'alpha', plot=False)
#     optimal_alpha_by_N.append(optimal_alpha)
#     # optimal_result = results_srf_inf[optimal_idx]
#     # np.save('data/srf/srf_{}ensembles_inflation_parametrized.npy'.format(N), np.array(results_srf_inf))
#     # np.save('data/srf/srf_{}ensembles_optimal.npy'.format(N), optimal_result)
# df = pd.DataFrame(errors, index=params_alpha, columns=params_N)
# df.to_csv('data/srf/srf_N_alpha_rmse.csv')

In [ ]:
%%time
# results = []
# params_alpha = []
# for k in range(5):
#     inf = 0.02 - 0.002 + 0.001*k
#     params_alpha.append(1+inf)
#     srf_inf = EnSRF(M, H, Q, R, y, x_0, P_0, N=50, alpha=1 + inf)
#     srf_inf.forward_estimation()
#     results.append(srf_inf.x)
# optimal_alpha, optimal_idx = estimate_error_with_params(true, results, params_alpha, 'alpha')
# np.save('data/srf/srf_optimal_50member.npy', results[optimal_idx])

### 観察
- $\alpha = 1.021$ 程度
- 実行時間: 
- rmse: 0.1679

### EnSRFまとめ
#### 最適結果:
1. $ N=20 $(速度重視)
  - $ \alpha: 1.12$
  - 実行時間: 23.9s
  - rmse: 0.2099
2. $ N = 40 $(精度重視)
  - $ \alpha: 1.021$
  - 実行時間: 53.2s
  - rmse: 0.16360
2. $ N = 50 $(精度重視)
  - $ \alpha: 1.021$
  - 実行時間: 2min2s
  - rmse: 0.1679

#### その他結果
- N=200, alpha=1.1で実行すると12min32sかかり1200stepあたりでdivergenceが起きていたがtrPはずっと高いままだった
- N=100, alpha=1.1で実行すると5min51sかかり1200stepあたりでdivergenceが起きていたがtrPはずっと高いままだった